# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2e5c0ec490>
├── 'a' --> tensor([[-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215]])
└── 'x' --> <FastTreeValue 0x7f2e5e172a60>
    └── 'c' --> tensor([[-0.7296, -0.2523,  0.5361, -0.2981],
                        [ 1.7676,  0.8522,  0.0807, -1.8366],
                        [-1.7261, -0.1644,  0.3017,  1.4700]])

In [4]:
t.a

tensor([[-0.1091,  0.5564, -1.5327],
        [ 0.8999, -0.0317,  1.5215]])

In [5]:
%timeit t.a

61.5 ns ± 0.0311 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2e5c0ec490>
├── 'a' --> tensor([[ 0.0260, -0.0870, -0.5371],
│                   [ 0.8383, -0.7053,  0.3026]])
└── 'x' --> <FastTreeValue 0x7f2e5e172a60>
    └── 'c' --> tensor([[-0.7296, -0.2523,  0.5361, -0.2981],
                        [ 1.7676,  0.8522,  0.0807, -1.8366],
                        [-1.7261, -0.1644,  0.3017,  1.4700]])

In [7]:
%timeit t.a = new_value

72 ns ± 0.133 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215]]),
    x: Batch(
           c: tensor([[-0.7296, -0.2523,  0.5361, -0.2981],
                      [ 1.7676,  0.8522,  0.0807, -1.8366],
                      [-1.7261, -0.1644,  0.3017,  1.4700]]),
       ),
)

In [10]:
b.a

tensor([[-0.1091,  0.5564, -1.5327],
        [ 0.8999, -0.0317,  1.5215]])

In [11]:
%timeit b.a

55.1 ns ± 0.0607 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.3863, -1.0418,  1.7694],
               [ 0.3511, -0.2696, -1.3766]]),
    x: Batch(
           c: tensor([[-0.7296, -0.2523,  0.5361, -0.2981],
                      [ 1.7676,  0.8522,  0.0807, -1.8366],
                      [-1.7261, -0.1644,  0.3017,  1.4700]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.0601 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

829 ns ± 0.103 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 40.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

136 µs ± 521 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2da43a0d30>
├── 'a' --> tensor([[[-0.1091,  0.5564, -1.5327],
│                    [ 0.8999, -0.0317,  1.5215]],
│           
│                   [[-0.1091,  0.5564, -1.5327],
│                    [ 0.8999, -0.0317,  1.5215]],
│           
│                   [[-0.1091,  0.5564, -1.5327],
│                    [ 0.8999, -0.0317,  1.5215]],
│           
│                   [[-0.1091,  0.5564, -1.5327],
│                    [ 0.8999, -0.0317,  1.5215]],
│           
│                   [[-0.1091,  0.5564, -1.5327],
│                    [ 0.8999, -0.0317,  1.5215]],
│           
│                   [[-0.1091,  0.5564, -1.5327],
│                    [ 0.8999, -0.0317,  1.5215]],
│           
│                   [[-0.1091,  0.5564, -1.5327],
│                    [ 0.8999, -0.0317,  1.5215]],
│           
│                   [[-0.1091,  0.5564, -1.5327],
│                    [ 0.8999, -0.0317,  1.5215]]])
└── 'x' --> <FastTreeValue 0x7f2da43a0b80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 94.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2da440e880>
├── 'a' --> tensor([[-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215],
│                   [-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215],
│                   [-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215],
│                   [-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215],
│                   [-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215],
│                   [-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215],
│                   [-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215],
│                   [-0.1091,  0.5564, -1.5327],
│                   [ 0.8999, -0.0317,  1.5215]])
└── 'x' --> <FastTreeValue 0x7f2daafee5e0>
    └── 'c' --> tensor([[-0.7296, -0.2523,  0.5361, -0.2981],
                        [ 1.7676,  0.8522,  0.0807, -1.8366],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.2 µs ± 65.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1091,  0.5564, -1.5327],
                [ 0.8999, -0.0317,  1.5215]],
       
               [[-0.1091,  0.5564, -1.5327],
                [ 0.8999, -0.0317,  1.5215]],
       
               [[-0.1091,  0.5564, -1.5327],
                [ 0.8999, -0.0317,  1.5215]],
       
               [[-0.1091,  0.5564, -1.5327],
                [ 0.8999, -0.0317,  1.5215]],
       
               [[-0.1091,  0.5564, -1.5327],
                [ 0.8999, -0.0317,  1.5215]],
       
               [[-0.1091,  0.5564, -1.5327],
                [ 0.8999, -0.0317,  1.5215]],
       
               [[-0.1091,  0.5564, -1.5327],
                [ 0.8999, -0.0317,  1.5215]],
       
               [[-0.1091,  0.5564, -1.5327],
                [ 0.8999, -0.0317,  1.5215]]]),
    x: Batch(
           c: tensor([[[-0.7296, -0.2523,  0.5361, -0.2981],
                       [ 1.7676,  0.8522,  0.0807, -1.8366],
                       [-1.7261, -0.1644,  0.3017,  1.4700]],
         

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215],
               [-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215],
               [-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215],
               [-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215],
               [-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215],
               [-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215],
               [-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215],
               [-0.1091,  0.5564, -1.5327],
               [ 0.8999, -0.0317,  1.5215]]),
    x: Batch(
           c: tensor([[-0.7296, -0.2523,  0.5361, -0.2981],
                      [ 1.7676,  0.8522,  0.0807, -1.8366],
                      [-1.7261, -0.1644,  0.3017,  1.4700],
                      [-0.7296, -0.2523,  0.5361, -0.2981],
                      [ 1.7676,  

In [28]:
%timeit Batch.cat(batches)

160 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 990 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
